In [0]:
import random
import re
import os
import tempfile
import cv2
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor
# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3
import keras
from keras.models import Model
from tensorflow.keras import layers
import tensorflow as tf


In [36]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

listing = os.listdir("./gdrive/My Drive/Mask_RCNN/videos/class_1")
count = 1
for file in listing:
    video = cv2.VideoCapture("./gdrive/My Drive/Mask_RCNN/videos/class_1/" + (file))
    print(video.isOpened())
    os.makedirs("./gdrive/My Drive/Mask_RCNN/videos/class_1/" + "class_1video_" + str(int(count)))
    while (video.isOpened()):
        frameId = video.get(1)
        success,image = video.read()
        # If condition for adjusting the frames in my particular video
        if(frameId >= 59 and frameId <= 119):
          if( success == True):
            image=cv2.resize(image,(224,224), interpolation = cv2.INTER_AREA)
            filename = "./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_" + str(int(count)) + "/image_" + str(int(frameId+1)) + ".jpg"
            print(filename)
            cv2.imwrite(filename,image)

          if (success != True):
            break     
        if(frameId > 119):
          break       
                              
    video.release()
    print('done')
    count+=1

cv2.destroyAllWindows()

True
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_60.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_61.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_62.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_63.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_64.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_65.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_66.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_67.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_68.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_69.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_70.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_71.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_72.jpg
./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/image_73.jpg
.

In [4]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 101kB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [0]:
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
import numpy as np
import glob,os
from scipy.misc import imread,imresize

batch_size = 16

In [38]:
#def bring_data_from_directory():
  datagen = ImageDataGenerator(rescale=1. / 255)
  train_generator = datagen.flow_from_directory(
          './gdrive/My Drive/Mask_RCNN/videos/class_1/',
          target_size=(224, 224),
          color_mode='rgb')
          #batch_size=batch_size,
          #class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          #shuffle=True,
          #classes=['CoverDrive','LegGlance','StraightDrive','PullShot','class_1video_1'])
"""
  validation_generator = datagen.flow_from_directory(
          'validate',
          target_size=(224, 224),
          color_mode='rgb',
          batch_size=batch_size,
          class_mode='categorical',  # this means our generator will only yield batches of data, no labels
          shuffle=True,
          classes=['CoverDrive','LegGlance','StraightDrive','PullShot','LoftedShot'])
  return train_generator,validation_generator
"""  

Found 61 images belonging to 1 classes.


"\nvalidation_generator = datagen.flow_from_directory(\n        'validate',\n        target_size=(224, 224),\n        color_mode='rgb',\n        batch_size=batch_size,\n        class_mode='categorical',  # this means our generator will only yield batches of data, no labels\n        shuffle=True,\n        classes=['CoverDrive','LegGlance','StraightDrive','PullShot','LoftedShot'])\nreturn train_generator,validation_generator\n"

In [39]:
#def load_ResNet50_model():
  base_model = tf.keras.applications.ResNet50(input_shape =(224,224,3), include_top = False, weights = 'imagenet')
  base_model.trainable = False
  last_layer = base_model.get_layer('conv5_block1_out').output
  x=tf.keras.layers.Flatten()(last_layer)
  model = tf.keras.Model(base_model.input , x)
  model.summary()
"""
  base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))
  base_model.trainable = False
  last_layer = base_model.get_layer('mixed9').output
  model = Model(inputs=base_model.input, outputs= last_layer)
  model.summary()
"""  
  
  #return model


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

"\nbase_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))\nbase_model.trainable = False\nlast_layer = base_model.get_layer('mixed9').output\nmodel = Model(inputs=base_model.input, outputs= last_layer)\nmodel.summary()\n"

In [40]:
x_feed = None   
c=1
check = (x_feed == None)
list_images = os.listdir("./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/")
for x in list_images: 
  print ("predict on image:",c)
  #img = cv2.imread("./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/" + (x), target_size = (224,224))
  original = image.load_img("./gdrive/My Drive/Mask_RCNN/videos/class_1/class_1video_1/" + (x), target_size=(224, 224))
  numpy_image = image.img_to_array(original)
  image_batch = np.expand_dims(numpy_image, axis=0)
  if (check):
    x_feed = model.predict(image_batch)
    check = False
  else:
    x_feed = np.append(x_feed, model.predict(image_batch),axis=0)
  c=c+1
  

predict on image: 1
predict on image: 2
predict on image: 3
predict on image: 4
predict on image: 5
predict on image: 6
predict on image: 7
predict on image: 8
predict on image: 9
predict on image: 10
predict on image: 11
predict on image: 12
predict on image: 13
predict on image: 14
predict on image: 15
predict on image: 16
predict on image: 17
predict on image: 18
predict on image: 19
predict on image: 20
predict on image: 21
predict on image: 22
predict on image: 23
predict on image: 24
predict on image: 25
predict on image: 26
predict on image: 27
predict on image: 28
predict on image: 29
predict on image: 30
predict on image: 31
predict on image: 32
predict on image: 33
predict on image: 34
predict on image: 35
predict on image: 36
predict on image: 37
predict on image: 38
predict on image: 39
predict on image: 40
predict on image: 41
predict on image: 42
predict on image: 43
predict on image: 44
predict on image: 45
predict on image: 46
predict on image: 47
predict on image: 48
p

In [41]:
x_feed.shape

(61, 100352)

In [0]:
#here you will have to make changes in dimensions according to the number of videos taken.
x_feed= x_feed.reshape(1,61,x_feed.shape[1])


In [43]:
x_feed.shape

(1, 61, 100352)

In [0]:
np.save(open('video_x_ResNet50.npy', 'wb'),x_feed)
#train_data = np.load(open('video_x_ResNet50.npy'),encoding= 'bytes')
train_data = pd.read_csv(open('video_x_ResNet50.npy'), encoding= 'utf-8')
train_data

In [0]:
def extract_features_and_store(train_generator,validation_generator,base_model):
  x_generator = None
  y_lable = None
  batch = 0
  for x,y in train_generator:
      if batch == (96/batch_size):
          break
      print "predict on batch:",batch
      batch+=1
      if x_generator==None:
         x_generator = base_model.predict_on_batch(x)
         y_lable = y
         print y
      else:
         x_generator = np.append(x_generator,base_model.predict_on_batch(x),axis=0)
         y_lable = np.append(y_lable,y,axis=0)
  x_generator,y_lable = shuffle(x_generator,y_lable)
  np.save(open('video_x_ResNet50.npy', 'w'),x_generator)
  np.save(open('video_y_ResNet50.npy','w'),y_lable)
  batch = 0
  x_generator = None
  y_lable = None
  for x,y in validation_generator:
      if batch == (96/batch_size):
          break
      print ("predict on batch validate:",batch)
      batch+=1
      if x_generator==None:
         x_generator = base_model.predict_on_batch(x)
         y_lable = y
      else:
         x_generator = np.append(x_generator,base_model.predict_on_batch(x),axis=0)
         y_lable = np.append(y_lable,y,axis=0)
  x_generator,y_lable = shuffle(x_generator,y_lable)
  np.save(open('video_x_validate_ResNet50.npy', 'w'),x_generator)
  np.save(open('video_y_validate_ResNet50.npy','w'),y_lable)

  train_data = np.load(open('video_x_ResNet50.npy'))
  train_labels = np.load(open('video_y_ResNet50.npy'))
  train_data,train_labels = shuffle(train_data,train_labels)
  validation_data = np.load(open('video_x_validate_ResNet50.npy'))
  validation_labels = np.load(open('video_y_validate_ResNet50.npy'))
  validation_data,validation_labels = shuffle(validation_data,validation_labels)

  train_data = train_data.reshape(train_data.shape[0],
                     train_data.shape[1] * train_data.shape[2],
                     train_data.shape[3])
  validation_data = validation_data.reshape(validation_data.shape[0],
                     validation_data.shape[1] * validation_data.shape[2],
                     validation_data.shape[3])
  
  return train_data,train_labels,validation_data,validation_labels

In [44]:
#def train_model(train_data,train_labels,validation_data,validation_labels):
  ''' used fully connected layers, SGD optimizer and 
      checkpoint to store the best weights'''
  
  model1 = Sequential()
  model1.add(LSTM(4096,dropout=0.2,input_shape= (x_feed.shape[1],x_feed.shape[2])))                  
  model1.add(Dense(1024, activation='relu'))
  model1.add(Dropout(0.5))
  model1.add(Dense(5, activation='softmax'))
  sgd = SGD(lr=0.00005, decay = 1e-6, momentum=0.9, nesterov=True)
  model1.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
  callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), ModelCheckpoint('video_1_LSTM_1_1024.h5', monitor='val_loss', save_best_only=True, verbose=0) ]
  nb_epoch = 32
  model1.fit(x_feed,x_label,batch_size=batch_size,nb_epoch=nb_epoch,callbacks=callbacks,shuffle=True,verbose=1)
  #model1.fit(train_data,train_labels,validation_data=(validation_data,validation_labels),batch_size=batch_size,nb_epoch=nb_epoch,callbacks=callbacks,shuffle=True,verbose=1)
  #return model1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


AttributeError: ignored

In [0]:
def test_on_whole_videos(train_data,train_labels,validation_data,validation_labels):
  parent = os.listdir("/Users/.../video/test")
  #.....................................Testing on whole videos.................................................................
  x = []
  y = []
  count = 0
  output = 0
  count_video = 0
  correct_video = 0
  total_video = 0
  base_model = load_InceptionV3_model()
  model = train_model(train_data,train_labels,validation_data,validation_labels)
  for video_class in parent[1:]:
      print video_class
      child = os.listdir("/Users/.../video/test" + "/" + video_class)
      for class_i in child[1:]:
          sub_child = os.listdir("/Users/.../video/test" + "/" + video_class + "/" + class_i)
          for image_fol in sub_child[1:]:
              if (video_class ==  'class_4' ):
                  if(count%4 == 0):
                      image = imread("/Users/.../video/test" + "/" + video_class + "/" + class_i + "/" + image_fol)
                      image = imresize(image , (224,224))

                      x.append(image)
                      y.append(output)
                      #cv2.imwrite('/Users/.../video/validate/' + video_class + '/' + str(count) + '_' + image_fol,image)
                  count+=1

              else:
                  if(count%4 == 0):
                      image = imread("/Users/.../video/test" + "/" + video_class + "/" + class_i + "/" + image_fol)
                      image = imresize(image , (224,224))
                      x.append(image)
                      y.append(output)
                      #cv2.imwrite('/Users/.../video/validate/' + video_class + '/' + str(count) + '_' + image_fol,image)
                  count+=1
          #correct_video+=1
          x = np.array(x)
          y = np.array(y)
          x_features = base_model.predict(x)
          #np.save(open('feat_' + 'class_' + str(output) + '_' + str(count_video) +'_'  + '.npy','w'),x)

          correct = 0
          
          answer = model.predict(x_features)
          for i in range(len(answer)):
              if(y[i] == np.argmax(answer[i])):
                  correct+=1
          print correct,"correct",len(answer)
          total_video+=1
          if(correct>= len(answer)/2):
              correct_video+=1
          x = []
          y = []
          count_video+=1
      output+=1

  print "correct_video",correct_video,"total_video",total_video
  print "The accuracy for video classification of ",total_video, " videos is ", (correct_video/total_video)

In [0]:
if __name__ == '__main__':
  train_generator,validation_generator = bring_data_from_directory()
  base_model = load_ResNet50_model()
  train_data,train_labels,validation_data,validation_labels = extract_features_and_store(train_generator,validation_generator,base_model)
  train_model(train_data,train_labels,validation_data,validation_labels)
  test_on_whole_videos(train_data,train_labels,validation_data,validation_labels)
